In [77]:
import pandas as pd
import os
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import glob
from typing import List, Dict, Tuple
from collections import Counter

In [98]:
def is_linkpost(row) -> bool:
    """Check if a post is marked as a linkpost by looking for 'this is a linkpost' at the beginning"""
    html_content = row.get('htmlBody')
    if pd.isna(html_content):
        return False
    
    try:
        # Parse HTML and get text content
        soup = BeautifulSoup(html_content, 'html.parser')
        text_content = soup.get_text().strip().lower()
        
        # Check if it starts with "this is a linkpost"
        return ('this is a linkpost') in text_content
    
    except Exception as e:
        print(f"Error checking linkpost status: {e}")
        return False

In [99]:
def clean_html(html_content: str) -> str:
    """Extract plain text from HTML, removing all tags and styling"""
    if pd.isna(html_content):
        return ''
    
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        # Get plain text, strip extra whitespace
        text = soup.get_text()
        # Clean up whitespace - replace multiple spaces/newlines with single spaces
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    except Exception as e:
        print(f"Error cleaning HTML: {e}")
        return ''

In [100]:
def extract_links_from_html(html_content: str) -> List[str]:
    """Extract all links from HTML content"""
    if pd.isna(html_content):
        return []
    
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        links = []
        
        # Find all anchor tags with href attributes
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            # Skip empty, anchor-only, or javascript links
            if href and not href.startswith('#') and not href.startswith('javascript:'):
                # Clean up relative URLs if needed
                if href.startswith('//'):
                    href = 'https:' + href
                elif href.startswith('/'):
                    href = 'https://lesswrong.com' + href
                links.append(href)
        
        return links
    except Exception as e:
        print(f"Error parsing HTML: {e}")
        return []

In [105]:
# Simple name-to-gender mapping (you can expand this)
import names
MALE_NAMES = names.MALE_NAMES
FEMALE_NAMES = names.FEMALE_NAMES

In [106]:
def extract_gender_from_username(username: str, display_name: str = None) -> str:
    """Attempt to extract gender from username and display name"""
    if pd.isna(username):
        return 'unknown'
    
    # Combine username and display name for analysis
    text_to_analyze = str(username).lower()
    if pd.notna(display_name):
        text_to_analyze += ' ' + str(display_name).lower()
    
    # Look for explicit gender indicators
    if any(word in text_to_analyze for word in ['girl', 'woman', 'female', 'lady', 'she', 'her', 'miss', 'madam', 'queen']):
        return 'female'
    if any(word in text_to_analyze for word in ['boy', 'man', 'male', 'guy', 'he', 'him', 'dude', 'bro', 'mr', 'mister', 'king']):
        return 'male'
    
    # Extract potential names from username (remove numbers, common separators)
    if any(word in text_to_analyze for word in MALE_NAMES):
        return 'male'
    if any(word in text_to_analyze for word in FEMALE_NAMES):
        return 'female'
    
    return username

In [107]:
def process_csv_file(filepath: str) -> None:
    """Process a single CSV file and add extracted links column"""
    try:
        df = pd.read_csv(filepath)
        
        # Add new columns for link information
        df['is_linkpost'] = df.apply(is_linkpost, axis=1)
        df['extracted_links'] = ''
        df['cleaned_htmlBody'] = ''
        df['user_gender'] = ''
        unknown_names = Counter()
        
        for idx, row in df.iterrows():
            # Clean the HTML content to plain text
            cleaned_text = clean_html(row.get('htmlBody'))
            df.at[idx, 'cleaned_htmlBody'] = cleaned_text
            
            # Extract gender from username
            gender = extract_gender_from_username(row.get('user.username'), row.get('user.displayName'))
            df.at[idx, 'user_gender'] = gender
            unknown_names[gender] += 1
            
            # Extract links from htmlBody
            html_links = extract_links_from_html(row.get('htmlBody'))
            
            # If it's a linkpost, skip the first link
            if row['is_linkpost'] and html_links:
                df.at[idx, 'extracted_links'] = '; '.join(html_links[1:])  # Skip first link
            else:
                df.at[idx, 'extracted_links'] = '; '.join(html_links)
        
        # Save back to the same file
        df.to_csv(filepath, index=False)
        
        # Return counts for summary
        posts_with_links = (df['extracted_links'] != '').sum()
        linkposts = df['is_linkpost'].sum()
        
        return posts_with_links, linkposts, unknown_names
    
    except Exception as e:
        print(f"Error processing {filepath}: {e}")
        return 0, 0, 0

In [ ]:
"""Main function to process all CSV files"""
base_path = "lw_csv"
total_posts_with_links = 0
total_linkposts = 0
files_processed = 0
unknown_names = Counter()

# Find all CSV files matching the pattern
csv_files = []
for year in range(2016, 2026):  # 2016 to 2025
    year_path = os.path.join(base_path, str(year))
    if os.path.exists(year_path):
        pattern = os.path.join(year_path, "*.csv")
        year_files = glob.glob(pattern)
        csv_files.extend(year_files)

print(f"Found {len(csv_files)} CSV files to process")

# Process each file and update it in place
for i, csv_file in enumerate(sorted(csv_files)):
    print(f"Processing {i+1}/{len(csv_files)}: {csv_file}")
    posts_with_links, linkposts, unknowns = process_csv_file(csv_file)
    unknown_names += unknowns
    
    total_posts_with_links += posts_with_links
    total_linkposts += linkposts
    files_processed += 1
    print(f"  ✓ Updated with {posts_with_links} posts with links, {linkposts} linkposts")

print(f"\nCompleted!")
print(f"Files processed: {files_processed}/{len(csv_files)}")
print(f"Total posts with extracted links: {total_posts_with_links}")
print(f"Total linkposts identified: {total_linkposts}")
print(f"\nEach CSV file now has these new columns added:")
print(f"  - 'is_linkpost': Boolean for linkpost detection")
print(f"  - 'extracted_links': Citation links (semicolon-separated)")
print(f"  - 'cleaned_htmlBody': Plain text with all HTML removed")

print(f'Unknown names: {len(unknown_names.keys())}')
for key, value in unknown_names.items():
    print(f'{key}: {value}')

Found 116 CSV files to process
Processing 1/116: lw_csv/2016/2016-01.csv
  ✓ Updated with 106 posts with links, 0 linkposts
Processing 2/116: lw_csv/2016/2016-02.csv
  ✓ Updated with 87 posts with links, 0 linkposts
Processing 3/116: lw_csv/2016/2016-03.csv
  ✓ Updated with 85 posts with links, 0 linkposts
Processing 4/116: lw_csv/2016/2016-04.csv
  ✓ Updated with 93 posts with links, 0 linkposts
Processing 5/116: lw_csv/2016/2016-05.csv
  ✓ Updated with 67 posts with links, 0 linkposts
Processing 6/116: lw_csv/2016/2016-06.csv
  ✓ Updated with 96 posts with links, 0 linkposts
Processing 7/116: lw_csv/2016/2016-07.csv
  ✓ Updated with 77 posts with links, 0 linkposts
Processing 8/116: lw_csv/2016/2016-08.csv
  ✓ Updated with 72 posts with links, 0 linkposts
Processing 9/116: lw_csv/2016/2016-09.csv
  ✓ Updated with 84 posts with links, 0 linkposts
Processing 10/116: lw_csv/2016/2016-10.csv
  ✓ Updated with 60 posts with links, 0 linkposts
Processing 11/116: lw_csv/2016/2016-11.csv
  ✓ 